<a href="https://colab.research.google.com/github/Surojit-KB/UNIV-AI-ML2-HW2/blob/main/Copy_of_Hw2_Part4_distribute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML-2: Trees, Model Interrogation and Bayesian Workflow
# Homework 2: Rossman Kaggle: Forecasting Sales
# Part 4 : Modelling with embeddings!
**ML-2 Cohort 1** <br>
**Instructor: Dr. Rahul Dave**<br>
**Max Score: 100** <br>
Name of Students:
1. Sukriti Paul
2. Harsh Goyal
3. Surojit Bhattacharyya

In [1]:
#importing libraries
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
%matplotlib inline

We will repeat the first initial steps again from Part 2

Lets import the feature_train_data.pickle file and set X,y values from the pickle file

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#your code here
with open("/content/drive/MyDrive/Colab Notebooks/UNIV-AI-ML2/HW/feature_train_data.pickle", 'rb') as f:
    (X,y) = pickle.load(f)

In [4]:
with open("/content/drive/MyDrive/Colab Notebooks/UNIV-AI-ML2/HW/result.pickle", 'rb') as f:
    result = pickle.load(f)

In [6]:
# we will split the train_ratio and 90% and 10% and set the train_size
train_ratio = 0.9
num_records = len(X)
train_size = int(train_ratio * num_records)

In [7]:
#lets look at our data
X[1], y[1]

(array([1058,    0,    0,    0,    1,    0,    0,    1]), '4491')

The next set of inputs is following: Write the code for this yourself

1. Do you want to one hot encode the data?
2. Do you want to provide embeddings as input - this will be set to True for models with entity embeddings
3. Do you want to save the emmbeddings? - again set to true if you want to entity embeddings
4. if 3 is set to true, we want to save them to a embeddings.pickle


In [8]:
#your code here
one_hot_as_input = False 
embeddings_as_input = True 
save_embeddings = True
saved_embeddings_fname = "embeddings.pickle" 

## Now lets work with Models with Entity embedding!!

Now that you have saved embeddings - push this into the other models as an input with X. 

How will we do this? 

We need to update our X values. 

1. We will define a function called embed_features, which will combine the embedding with X. 
2. Call this function and update it with the inital X values taken from the pickle file - features_train_data
3. Then split you data, X_emb - into Xtrain and X_Val, y_train and y_val remain the same
4. Sample the data

In [9]:
#call our saved embeddings from part 3
saved_embeddings_fname = "/content/drive/MyDrive/Colab Notebooks/UNIV-AI-ML2/HW/embeddings.pickle"

In [10]:
f_embeddings = open(saved_embeddings_fname, "rb")
embeddings = pickle.load(f_embeddings)

In [14]:
for embed in embeddings:
  print(embed.shape)

(1115, 10)
(7, 6)
(3, 2)
(12, 6)
(31, 10)
(12, 6)


In [12]:
data_order=['store','year','month','day','dow','open','promo','state']
embeddings_order=['store','dow','year','month','day','state']

In [13]:
#combining embedding
def embed_features(X, saved_embeddings_fname):
    f_embeddings = open(saved_embeddings_fname, "rb")
    embeddings = pickle.load(f_embeddings)

    index_embedding_mapping = {0: 0, 1: 2, 2: 3, 3: 4, 4: 1, 7: 5}
    X_embedded = []

    (num_records, num_features) = X.shape
    for record in X:
        embedded_features = []
        for i, feat in enumerate(record):
            feat = int(feat)
            if i not in index_embedding_mapping.keys():
                embedded_features += [feat]
            else:
                embedding_index = index_embedding_mapping[i]
                embedded_features += embeddings[embedding_index][feat].tolist()

        X_embedded.append(embedded_features)

    return np.array(X_embedded)

**Explain what is happening the function above**

Here we are replacing our data values by there respective embeddings. If we take an embedding, for example, the Store index embedding, it has a shape of (1115,10) which means for each of the 1115 store indexes, it has a vector of 10 as the embedding. This is the vector that we call as the embedding and we want to use it to replace the corresponding data value. So, we are first going to the correct feature embedding by giving the embedding index correctly and then going to the correct data value by providing the data value as the further index and converting the embedding to list and setting it as the data value.<br>

### Embedding with X - input

In [15]:
#check if embedding is needed, if yes call embed_features - with X and the embeddings passed to it - call this new X as X_emb
if embeddings_as_input:
  #your code here 
  X_emb=embed_features(X,saved_embeddings_fname)

Split the train and validation based on train size and on the new X_emb values from the previous code

In [16]:
#update the X_train and X_val
#your code here 
X_train=X_emb[:train_size,:]
X_val=X_emb[train_size:,:]
y_train=y[:train_size]
y_train=y_train.astype(np.int)
y_val=y[train_size:]
y_val=y_val.astype(np.int)

In [17]:
def sample(X, y, n):
    '''random samples'''
    num_row = X.shape[0]
    indices = np.random.randint(num_row, size=n)
    return X[indices, :], y[indices]

In [18]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("Number of samples used for training: " + str(y_train.shape[0]))

Number of samples used for training: 200000


## Add the embeddings into the Models and check their MAPE!

All the models defined here will have the same parameters as the ones defined in Part 2!

In [19]:
#define MAPE Function here
#your code here
def MAPE(Y_actual,Y_Predicted):
    #your code here
    return np.mean(np.abs((Y_actual-Y_Predicted)/Y_actual))

### Model 1: Random Forests

In [21]:
#LOG TRANSFORMING Y VALUES AND THEN SCALING THEM
y_train=np.log(y_train)
y_val=np.log(y_val)
y_max=max(np.max(y_train),np.max(y_val))
y_train=y_train/y_max
y_val=y_val/y_max

In [22]:
#Using log scaled target values 
model1=RandomForestRegressor(n_estimators = 200, max_depth = 35, min_samples_split = 2, min_samples_leaf = 1,n_jobs=-1,verbose=1)
model1.fit(X_train,y_train)
y_pred=model1.predict(X_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  5.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    2.0s finished


In [23]:
print('MAPE score is:',MAPE(y_val,y_pred))

MAPE score is: 0.012296595383149252


In [24]:
result['With EE']['Random Forest']=MAPE(y_val,y_pred)

### Model 2: Boosting

In [25]:
#your code here
dtrain=xgb.DMatrix(X_train,label=y_train)
num_round=3000
param={'nthread': -1,'max_depth': 7,'eta': 0.02,'silent': 1,'objective': 'reg:linear','colsample_bytree': 0.7,'subsample': 0.7}
best=xgb.train(param,dtrain,num_round)

In [26]:
feature_Xtr = xgb.DMatrix(X_train)
feature_Xval = xgb.DMatrix(X_val)
ytrain_pred=best.predict(feature_Xtr)
yval_pred=best.predict(feature_Xval)

In [27]:
print('Train MAPE score is:',MAPE(y_train,ytrain_pred))
print('Validation MAPE score is:',MAPE(y_val,yval_pred))

Train MAPE score is: 0.0063567874485113635
Validation MAPE score is: 0.01289825014905645


In [28]:
result['With EE']['Gradient Boosted Tree']=MAPE(y_val,yval_pred)

In [35]:
df=pd.DataFrame.from_records(result)
df

,With EE,Without EE
Neural Network,0.011585,0.011897
Random Forest,0.012297,0.026278
Gradient Boosted Tree,0.012898,0.020258


In [40]:
#Lets convert the dataframe to makrdown to show it in the final result
df.to_markdown()

'|                       |   With EE |   Without EE |\n|:----------------------|----------:|-------------:|\n| Neural Network        | 0.0115853 |    0.011897  |\n| Random Forest         | 0.0122966 |    0.0262777 |\n| Gradient Boosted Tree | 0.0128983 |    0.0202583 |'

In [33]:
with open("/content/drive/MyDrive/Colab Notebooks/UNIV-AI-ML2/HW/result.pickle", 'wb') as f:
    pickle.dump((result),f)

## Final Commments!

Apart from how long this homework was, lets make some other final comments

**Question 1: Did you models with Entity Embeddings perfom better?**

ANS: Yes the models with Entity Embeddings performed better than the models which were provided with the Dataset directly. We can see that in the table above, all the models have a lower MAPE score when provided with embeddings as input. Specifically, random forest, whcih was the worst perfoming model amongst the models without EE performs reasonably well( even better than Gradient Boosted Trees) when provided with the embeddings. Neural Nets have a very slight improvement but the other two models show a clear decrease in the MAPE score.

**Question 2: Now that you have completed this homework, lets answer the main purpose of the homework - How do you think entity embeddings improved the MAPE score. To show this do a similar table like the one did in Paper**

ANS: 


```
|       method          |   With EE |   Without EE |
|:----------------------|----------:|-------------:|
| Neural Network        | 0.0115853 |    0.011897  |
| Random Forest         | 0.0122966 |    0.0262777 |
| Gradient Boosted Tree | 0.0128983 |    0.0202583 |
```
We can see the change in the MAPE scores for both the ensemble models quite clearly. Random Forest especially looks like the big winner in the EE case with a quite large change in the MAPE score. Actually, we had got better MAPE score for Random Forest without the use of one-hot encoding, but we have used the result from the one-hot encoded dataset here to keep the uniformity between all the three models. Also, we have used log-scaled target variable(*Sales*) in all the the three models.




**Question 3: Add a table here to show the similar to the one done in paper - to show the different MAPE values with and without embeddings**

ANS:
 


```
|       method          |   With EE |   Without EE |
|:----------------------|----------:|-------------:|
| Neural Network        | 0.0115853 |    0.011897  |
| Random Forest         | 0.0122966 |    0.0262777 |
| Gradient Boosted Tree | 0.0128983 |    0.0202583 |
```



THE END<br>
Thanks for the read.